## Test - EPA HUB

In [37]:
import requests
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup

### Parameters

In [4]:
URL = "https://www.epa.gov/climateleadership/ghg-emission-factors-hub"

### Functions

In [6]:
headers = {
    "Authority": "www.epa.gov",
    "Method": "GET",
    "Path": "/climateleadership/ghg-emission-factors-hub",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "If-None-Match": "1689275341",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


### Tests

In [45]:
r = requests.get(URL, headers=headers)
soup = BeautifulSoup(r.content, "html.parser")
# Get all hrefs and their text
href_dicts = []
hrefs = soup.find_all("a")
for href in hrefs:
    href_dict = {}
    text=  href.text
    url = href.get("href")

    if "GHG" not in text:
        continue

    if "xls" in url:     
        href_dict["name"] = str(text).replace("\n", "")
        href_dict["url"] =  f"https://www.epa.gov{url}"
        href_dicts.append(href_dict)
 
    
    elif "pdf" in url:
        href_dict["name"] = str(text).replace("\n", "") 
        href_dict["url"] = f"https://www.epa.gov{url}"

        href_dicts.append(href_dict)

# Create dataframe
df = pd.DataFrame.from_dict(href_dicts)

In [46]:
df

,name,url
0,2023 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
1,2023 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
2,ARCHIVED 2022 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
3,ARCHIVED 2022 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
4,ARCHIVED 2021 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
5,ARCHIVED 2021 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
6,ARCHIVED 2020 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
7,ARCHIVED 2020 GHG Emission Factors Hub (pdf),https://www.epa.gov/sites/default/files/2021-0...
8,ARCHIVED 2018 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
9,ARCHIVED 2018 GHG Emission Factors Hub (pdf),https://www.epa.gov/sites/default/files/2018-0...


### Getting xlsx files

In [47]:
xlsx_urls = df["url"].tolist()
undesired_words = ["pdf"]

xlsx_urls = [url for url in xlsx_urls if not any(word in url for word in undesired_words)]
xlsx_urls

['https://www.epa.gov/system/files/documents/2023-03/ghg-emission-factors-hub.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2022.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2021.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2020.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2018.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2015.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2014.xlsx',
 'https://www.epa.gov/system/files/documents/2023-04/ghg-emission-factors-hub-2011.xlsx']

In [115]:
mega_df = pd.DataFrame([])
for xlsx_url in xlsx_urls[:1]:
    response = requests.get(xlsx_url, headers=headers)
    xls_file = pd.ExcelFile(BytesIO(response.content))
    # Create a dictionary of DataFrames, with sheet name as key
    dataframes = {sheet_name: xls_file.parse(sheet_name) for sheet_name in xls_file.sheet_names}
    key_of_interest = None
    for key in dataframes.keys():
        if "Hub" in key:
            key_of_interest = key

    if key_of_interest is None:
        print(f"Skipping {xlsx_url}")
        continue

    ghg_data = dataframes[key_of_interest]
    max_nans = 8  # Max number of NaNs allowed
    ghg_data = ghg_data.dropna(thresh=ghg_data.shape[1] - max_nans + 1)
    # ghg_data = ghg_data.dropna(axis=1, how="all")
    # Remove rows with several NaNs
    # ghg_data = ghg_data.dropna(axis=0, thresh=2)

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [116]:
ghg_data.head(40)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
12,NaN,NaN,Fuel Type,Heat Content (HHV),CO2 Factor,CH4 Factor,N2O Factor,CO2 Factor,CH4 Factor,N2O Factor
13,NaN,NaN,NaN,mmBtu per short ton,kg CO2 per mmBtu,g CH4 per mmBtu,g N2O per mmBtu,kg CO2 per short ton,g CH4 per short ton,g N2O per short ton
15,NaN,NaN,Anthracite Coal,25.09,103.69,11,1.6,2602,276,40
16,NaN,NaN,Bituminous Coal,24.93,93.28,11,1.6,2325,274,40
17,NaN,NaN,Sub-bituminous Coal,17.25,97.17,11,1.6,1676,190,28
18,NaN,NaN,Lignite Coal,14.21,97.72,11,1.6,1389,156,23
19,NaN,NaN,Mixed (Commercial Sector),21.39,94.27,11,1.6,2016,235,34
20,NaN,NaN,Mixed (Electric Power Sector),19.73,95.52,11,1.6,1885,217,32
21,NaN,NaN,Mixed (Industrial Coking),26.28,93.9,11,1.6,2468,289,42
22,NaN,NaN,Mixed (Industrial Sector),22.35,94.67,11,1.6,2116,246,36


In [104]:
ghg_data.to_csv("epa-hub.csv", index=False)